In [0]:
import json

def initializeDivisor(vertices):
        configuration = {}
        for i in range(len(vertices)):
            configuration[i] = 0
        return configuration

def parseJSON(graphJSON):
    graphInfo = json.loads(graphJSON)
    G = Graph()
    G.add_vertices(range(len(graphInfo['nodes'])))

    div = initializeDivisor(G.vertices())

    for i in range(len(graphInfo['nodes'])):
        if graphInfo['nodes'][i]['text'] == "":
            div[i] = 0
        else:
            div[i] = int(graphInfo['nodes'][i]['text'])


    edges = graphInfo['links']
    for edge in edges:
        G.add_edge(edge["nodeA"],edge["nodeB"])

    return GraphConfig(G,div)

class GraphConfig:
    def __init__(self,graph,div):
        self.graph = graph
        self.div = div
        self.burn = self.createBurnDict(graph.vertices())
        self.vertices = graph.vertices()

    def createBurnDict(self,vertices):
        burning = {}
        for i in range(len(vertices)):
            burning[i] = False
        return burning

    def chipFire(self,vertex):
        neighborhood = self.graph.neighbors(vertex)
        for neighbor in neighborhood:
            self.div[neighbor] += 1
            self.div[vertex] -= 1

    def isGonnaBurn(self,vertex):
        burnValence = 0
        neighborhood = self.graph.neighbors(vertex)
        for neighbor in neighborhood:
            if self.burn[neighbor]:
                burnValence += 1
        if self.div[vertex] < burnValence:
            return True
        else:
            return False

    def startFire(self):
        for vert in self.vertices:
            if self.div[vert] < 0:
                self.burn[vert] = True
            else:
                self.burn[vert] = False

    def isEffectiveDivisor(self):
        toRtn = True
        for vert in self.vertices:
            if self.div[vert] < 0:
                toRtn = False
        return toRtn

    def burningRound(self):
        for vertex in self.vertices:
            if self.isGonnaBurn(vertex):
                self.burn[vertex] = True

    def setFire(self,vertex):
        self.burn[vertex] = True

    def getCurrentBurn(self):
        toRtn = []
        for vert in self.vertices:
            if self.burn[vert]:
                toRtn.append(vert)
        return toRtn

    def getUnburnt(self):
        toRtn = []
        for vert in self.vertices:
            if not self.burn[vert]:
                toRtn.append(vert)
        return toRtn

def dharsBurn(GC):
    toRtn = True
    while True:
        oldBurn = GC.getCurrentBurn()
        GC.burningRound()
        newBurn = GC.getCurrentBurn()
        existsNewVert = False

        for vert in GC.vertices:
            if (vert in newBurn) and (not vert in oldBurn):
                existsNewVert = True

        if GC.getCurrentBurn() == GC.vertices:
            toRtn = False
            break
        elif existsNewVert:
            continue
        elif GC.isEffectiveDivisor():
            break
        else:
            unburnt = GC.getUnburnt()
            for vert in unburnt:
                GC.chipFire(vert)
            GC.startFire()
    return toRtn

def main():
    print("Please Input your graph JSON")
    graphJSON = raw_input()
    GC = parseJSON(graphJSON)
    GC.graph.show()
    GC.startFire()
    success = dharsBurn(GC)
    if success:
        print("This divisor is effective")
    else:
        print("This divisor is not effective")

main()